<a href="https://colab.research.google.com/github/MarieChui/github_action/blob/main/Infos_g%C3%A9n%C3%A9rales_banque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import des packages

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


2. Chargement de la page d'accueil de Trustpilot sur le domaine ATM

In [2]:
url = 'https://www.trustpilot.com/categories/ATM'
reponse = requests.get(url)
soup = BeautifulSoup(reponse.text, "html.parser")

In [5]:
elements = []
#On crée une boucle pour récupérer les infos sur chaque page
items = soup.findAll('div', attrs ={'class':"paper_paper__29o4A paper_outline__3rnlh card_card__2F_07 card_noPadding__1tkWv styles_wrapper__2QC-c"})
  #On récupère la position du nom, trustcore+reviews et le domaine
lien_etab = []
for item in items:
  nom = item.find('p', attrs ={'class':"typography_heading-xs__D8ZZo typography_appearance-default__3vWd5 styles_displayName__1LIcI"})
  tru = item.find('p', attrs ={'class':"typography_body-m__3GWSM typography_appearance-subtle__2GxHy styles_ratingText__nheL7"})
  lien_note_item = item.find('a', attrs ={'class':"link_internal__YpiJI link_wrapper__LEdx5 styles_linkWrapper__3x9X7"}, href=True)
  lien_note= 'https://www.trustpilot.com'+lien_note_item['href']
  reponse_t = requests.get(lien_note)
  soup_t = BeautifulSoup(reponse_t.text, "html.parser")
  items_t = soup_t.findAll('p', attrs ={'class':"typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_cell__qnPHy styles_percentageCell__cHAnb"})
  notes = []
  for item_t in items_t:
    note = int(item_t.text.replace("%","").replace("<",""))/100
    notes.append(note)
    #Il arrive que l'établissement n'ait pas de truscore/review
  if tru is not None:
    a = tru.text.split()[1]
    tru = float(a.split('|')[0])
    rev = int(a.split('|')[-1].replace(",",""))
  else:
    tru = 0.0
    rev = 0
  element = {"Nom":nom.text, "Trustcore":tru, "Reviews":rev, "notes":notes}
  a=0
    #Condition pour avoir des éléments uniques dans la liste des établissements
  if len(elements)==0:
    elements.append(element)
  else:
    for i in range(len(elements)):
      if elements[i]["Nom"] != element["Nom"]:
        a+=0
      else:
          a+=1
    if a==0:
      elements.append(element)
elements

[{'Nom': 'Liberty First Credit Union',
  'Trustcore': 4.8,
  'Reviews': 134,
  'notes': [0.91, 0.04, 0.03, 0.01, 0.01]},
 {'Nom': 'Crypto Dispensers',
  'Trustcore': 3.6,
  'Reviews': 74,
  'notes': [0.64, 0.11, 0.04, 0.01, 0.2]},
 {'Nom': 'Cushion',
  'Trustcore': 4.7,
  'Reviews': 2787,
  'notes': [0.92, 0.03, 0.01, 0.01, 0.03]},
 {'Nom': 'Heritage Valley Federal Credit Union',
  'Trustcore': 4.1,
  'Reviews': 11,
  'notes': [0.82, 0.0, 0.09, 0.09, 0.0]},
 {'Nom': 'Kryptominerstech',
  'Trustcore': 4.0,
  'Reviews': 4,
  'notes': [1.0, 0.0, 0.0, 0.0, 0.0]},
 {'Nom': 'cryptobaseatm.com',
  'Trustcore': 4.0,
  'Reviews': 3,
  'notes': [1.0, 0.0, 0.0, 0.0, 0.0]},
 {'Nom': 'Slide2Thrive',
  'Trustcore': 0.0,
  'Reviews': 0,
  'notes': [0.0, 0.0, 0.0, 0.0, 0.0]},
 {'Nom': 'Meriwest Credit Union',
  'Trustcore': 3.2,
  'Reviews': 4,
  'notes': [0.75, 0.0, 0.0, 0.0, 0.25]},
 {'Nom': 'Coinhub Bitcoin ATM',
  'Trustcore': 3.2,
  'Reviews': 1,
  'notes': [0.0, 0.0, 0.0, 0.0, 1.0]},
 {'Nom': 'A

Boucle pour avoir chaque le pourcentage de chaque note dans les dictionnaires


In [6]:
for i in range(len(elements)):
  a=0
  for j in range(len(elements[i]['notes'])):
    if a == 0:
      elements[i]['Excellent']=elements[i]['notes'][j]
    if a == 1:
      elements[i]['Great']=elements[i]['notes'][j]
    if a == 2:
      elements[i]['Average']=elements[i]['notes'][j]
    if a == 3:
      elements[i]['Poor']=elements[i]['notes'][j]
    if a == 4:
      elements[i]['Bad']=elements[i]['notes'][j]
    a+=1
  elements[i].pop('notes')

In [7]:
#elements_array=np.array(elements)
df = pd.DataFrame(elements, index=[i for i in range(1,len(elements)+1)]) # Le nom des colonnes
df

,Nom,Trustcore,Reviews,Excellent,Great,Average,Poor,Bad
1,Liberty First Credit Union,4.8,134,0.91,0.04,0.03,0.01,0.01
2,Crypto Dispensers,3.6,74,0.64,0.11,0.04,0.01,0.20
3,Cushion,4.7,2787,0.92,0.03,0.01,0.01,0.03
4,Heritage Valley Federal Credit Union,4.1,11,0.82,0.00,0.09,0.09,0.00
5,Kryptominerstech,4.0,4,1.00,0.00,0.00,0.00,0.00
6,cryptobaseatm.com,4.0,3,1.00,0.00,0.00,0.00,0.00
7,Slide2Thrive,0.0,0,0.00,0.00,0.00,0.00,0.00
8,Meriwest Credit Union,3.2,4,0.75,0.00,0.00,0.00,0.25
9,Coinhub Bitcoin ATM,3.2,1,0.00,0.00,0.00,0.00,1.00
10,Asic Miners Rig,2.9,10,0.40,0.00,0.00,0.00,0.60


In [8]:
len(elements)

17

In [9]:
df.to_csv("Infos_ATM.csv", sep = ',')

In [10]:
from google.colab import files
files.download("Infos_ATM.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>